## Training of models predicting insertions only from the data of the two neighboring positions

In [1]:
#import packages
import pickle
import numpy as np
from Bio import SeqIO
from Bio.PDB import *
from Bio.PDB import PDBParser
import seaborn as sns 
from utils.plotting import *
from utils.processing import *
from utils.model import *
from Bio import SeqIO
import pickle
import scipy.stats
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model

# set styles
plt.style.use('./utils/domain_ins.mplstyle')
plt.rcParams['svg.fonttype'] = 'none'
print('done')

/camp/home/acarb/.conda/envs/DIscreen_env/lib/python3.7/site-packages/Bio/SubsMat/__init__.py:131: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  BiopythonDeprecationWarning,


done


In [2]:
# define dirs
base = '/camp/lab/debenedictise/home/users/acarb/gitfolder/DI_screen_MathonyDebug'
in_folder = f'{base}/analysis/input_data'
data_folder = f'{base}/analysis/output_datasets'
fig_folder = f'{base}/analysis/figures'
fasta_sequences = {rec.id : rec.seq for rec in SeqIO.parse(f'{in_folder}/proteins.fasta', 'fasta')}

# import data
with open(f'{data_folder}/proteins_training.pickle', 'rb') as input:
    full_dataset = pickle.load(input)
input.close()
print(full_dataset['AraC']['2'])

with open(f'{data_folder}/analysis_dict.pickle', 'rb') as input:
    analysis_dict = pickle.load(input)
input.close()
print('done')

           log    prox_AAs Sec_structure       ASA   pLDDT  Linker_idx_Suyama  \
0          NaN      [0, 1]             C  0.981132  36.965             0.1085   
1    -1.716304      [1, 2]             C  0.970823  37.655            -0.0930   
2     2.193842      [2, 3]             C  0.843464  42.420            -0.0930   
3     0.229976      [3, 4]             C  0.517915  51.380            -0.0655   
4     0.968363      [4, 5]             C  0.578154  54.540            -0.0230   
5     0.243837      [5, 6]             C  0.536712  58.655             0.0215   
6     0.091946      [6, 7]             C  0.203582  62.890            -0.2310   
7    -3.032138      [7, 8]             C  0.147956  60.550            -0.1700   
8    -3.309773      [8, 9]             C  0.271341  62.665             0.1380   
9    -0.174089     [9, 10]             C  0.652798  62.885            -0.1700   
10   -7.431530    [10, 11]             C  0.853291  55.880            -0.0735   
11   -5.108909    [11, 12]  

# Classifiers

### Individual proteins

Binerize the output:

In [3]:
for name, protein in full_dataset.items():
    for key, dataset in protein.items():
        dataset.loc[dataset.log>0, 'log']= 1
        dataset.loc[dataset.log<=0, 'log'] = 0
pd.set_option('display.max_rows', None)

print(dataset)

     log    prox_AAs Sec_structure       ASA   pLDDT  Linker_idx_Suyama  \
0    NaN      [0, 1]             C  0.957692  52.690             0.0490   
1    1.0      [1, 2]             C  0.957692  60.775            -0.1445   
2    1.0      [2, 3]             C  0.849057  69.485            -0.0850   
3    1.0      [3, 4]             C  0.849057  78.580             0.0400   
4    1.0      [4, 5]             C  0.942073  81.825             0.1380   
5    1.0      [5, 6]             C  0.832317  82.810             0.0130   
6    1.0      [6, 7]             C  0.800958  86.025             0.1095   
7    1.0      [7, 8]             C  0.910714  90.390             0.1395   
8    1.0      [8, 9]             C  0.729839  93.590            -0.0260   
9    1.0     [9, 10]             C  0.551925  95.400             0.0080   
10   1.0    [10, 11]             C  0.410918  96.790             0.1280   
11   1.0    [11, 12]             C  0.353164  97.420             0.1335   
12   1.0    [12, 13]     

Build classifier:

**AraC**

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
GBC = Classifier(full_dataset['AraC']['2'], n_cross_val=5, title_text='AraC_model_context-2')
GBC.encode_AA_sequences(fasta_sequences['AraC_s']) #hot-encoded
GBC.encode_sec_structures() #hot-encoded
del GBC.dataset['prox_AAs']
GBC.clf = GradientBoostingClassifier(n_estimators=150, criterion='friedman_mse', learning_rate=.2, subsample=1, max_depth=3, min_samples_split=4, max_features='sqrt', loss='exponential', random_state=42) 
model_AraC, X_test_AraC, y_test_AraC, X_train_AraC, y_train_AraC, scores, kf_Ara, scaler = GBC.split_and_train() 

GBC.metrics() # plots ROC and precision vs recall plots
print(GBC.dataset)


     A  C  D  E  F  G  H  I  K  L  M  N  P  Q  R  S  T  V  W  Y  index  log  \
0    1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0      0  NaN   
1    1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0      1  0.0   
2    1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0      2  1.0   
3    1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0      3  1.0   
4    0  0  0  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0      4  1.0   
5    0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0      5  1.0   
6    0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0      6  1.0   
7    0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0  0      7  0.0   
8    0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0      8  0.0   
9    0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0  0      9  0.0   
10   0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0     10  0.0   
11   0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 

     Coil  Sheet  Helix  A  C  D  E  F  G  H  I  K  L  M  N  P  Q  R  S  T  V  \
1       1      0      0  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
2       1      0      0  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
3       1      0      0  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0   
4       1      0      0  0  0  0  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0   
5       1      0      0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0   
6       1      0      0  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0   
7       1      0      0  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0   
8       1      0      0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0   
9       1      0      0  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0   
10      1      0      0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0   
11      1      0      0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0   
12      1      0      0  0  

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


Mean AP is: 0.8162868504044974


findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


     Coil  Sheet  Helix  A  C  D  E  F  G  H  I  K  L  M  N  P  Q  R  S  T  V  \
1       1      0      0  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
2       1      0      0  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
3       1      0      0  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0   
4       1      0      0  0  0  0  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0   
5       1      0      0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0   
6       1      0      0  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0   
7       1      0      0  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0   
8       1      0      0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0   
9       1      0      0  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0   
10      1      0      0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0   
11      1      0      0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0   
12      1      0      0  0  

**TVMV**

In [ ]:
GBC = Classifier(full_dataset['TVMV']['2'], n_cross_val=5, title_text='TVMV_model_context-2')
GBC.encode_AA_sequences(fasta_sequences['TVMV_s'])
GBC.encode_sec_structures()
del GBC.dataset['prox_AAs']
model_TVMV, X_test_TVMV, y_test_TVMV, X_train_TVMV, y_train_TVMV, scores, kf_TVMV, scaler = GBC.split_and_train()
GBC.metrics()

In [ ]:
GBC = Classifier(full_dataset['Flp']['2'], n_cross_val=5, title_text='Flp_model_context-2')
GBC.encode_AA_sequences(fasta_sequences['Flp_s'])
GBC.encode_sec_structures()
del GBC.dataset['prox_AAs']
model_Flp, X_test_Flp, y_test_Flp, X_train_Flp, y_train_Flp, scores, kf_Flp, scaler = GBC.split_and_train()
GBC.metrics()

In [ ]:
GBC = Classifier(full_dataset['SigF']['2'], n_cross_val=5, title_text='SigF_model_context-2')
GBC.encode_AA_sequences(fasta_sequences['SigF_s'])
GBC.encode_sec_structures()
del GBC.dataset['prox_AAs']
model_SigF, X_test_SigF, y_test_SigF, X_train_SigF, y_train_SigF, scores, kf_SigF, scaler = GBC.split_and_train()
GBC.metrics()

## Prepare complete dataset

Add output from transformer insertion model:

Combine datasets to one:

In [ ]:
final_dataset = full_dataset['AraC'].copy()
original_column_order = final_dataset['2'].columns.tolist()

fasta_sequences_concat = fasta_sequences['AraC_s'] + fasta_sequences['TVMV_s'] + fasta_sequences['Flp_s'] + fasta_sequences['SigF_s']
for protein in ['TVMV', 'Flp', 'SigF']:
    for idx, dataset in full_dataset[f'{protein}'].items():
        final_dataset[idx] = pd.concat([final_dataset[idx], dataset], axis=0)
        final_dataset[idx] = final_dataset[idx].reset_index(drop=True)     
        final_dataset[idx] = final_dataset[idx][original_column_order]
        del final_dataset[idx]['prox_AAs']
        
# Step 3: Reorder columns to their original order

print(final_dataset)

Train model:

In [ ]:
random_forest = Classifier(final_dataset['2'], n_cross_val=5, title_text='Full_dataset_context-2')
random_forest.encode_AA_sequences(fasta_sequences_concat)
random_forest.encode_sec_structures()
#random_forest.grid_search({"alpha": [.00001, .0001, .001], 'penalty':['l1'], 'tol':[1e-3, 5e-4, 1e-4]}, linear_model.Perceptron())
model_complete, X_test_complete, y_test_complete, X_train_complete, y_train_complete, scores_complete, kf_complete, scaler_complete = random_forest.split_and_train()
random_forest.metrics()
scores_complete['test_recall'].mean().round(3)

Train model on AA one-hot-encoding alone:

In [ ]:
print(final_dataset['2']['log'])
GBC_one_hot = Classifier(final_dataset['2']['log'], n_cross_val=5, title_text='Full_dataset_AA')
GBC_one_hot.encode_AA_sequences(fasta_sequences_concat)
model_AA, X_test_AA, y_test_AA, X_train_AA, y_train_AA, scores_AA, kf_AA, scaler_AA = GBC_one_hot.split_and_train()
GBC_one_hot.metrics()
scores_AA['test_recall'].mean().round(3)

### Analyze feature importance

In [ ]:
print(final_dataset['2'])
GBC = Classifier(final_dataset['2'], n_cross_val=5, title_text='Model trained without AA sequence')
GBC.encode_AA_sequences(fasta_sequences_concat) #it says without that's why I commented it
GBC.encode_sec_structures()
model_complete, X_test_complete, y_test_complete, X_train_complete, y_train_complete, scores_complete, kf_complete, scaler_complete = GBC.split_and_train()
GBC.metrics()
scores_complete['test_average_precision']

In [ ]:
from sklearn.inspection import permutation_importance
np.set_printoptions(threshold=np.inf)
GBC.clf.fit(X_train_complete, y_train_complete)
feature_importance = GBC.clf.feature_importances_
sorted_idx = np.argsort(feature_importance)
#print(sorted_idx)
#print(GBC.dataset.columns)
#print(GBC.dataset.columns[sorted_idx])

pos = np.arange(sorted_idx.shape[0]) + 0.5
fig = plt.figure(figsize=(12, 14))
plt.subplot(1, 2, 1)
data = pd.DataFrame([np.array(GBC.dataset.columns)[sorted_idx], feature_importance[sorted_idx]]).T
data.columns=['features', 'importance']
data.features = data.features.str.replace('_', ' ')
#plt.barh(pos, feature_importance[sorted_idx], align="center")
g = sns.barplot(data= data.iloc[::-1], y='features', x='importance', color='grey', linewidth=1.5, capsize=.2, errcolor="black", edgecolor="black")
g.set(xlim=(0, .25))
plt.ylabel('')
plt.xlabel('Decrease in impurity')
plt.title("Feature Importance (Mean Decrease in Impurity)")
result = permutation_importance(GBC.clf, X_train_complete, y_train_complete, n_repeats=10, random_state=42, n_jobs=None)

#result = permutation_importance(GBC.clf, X_test_complete, y_test_complete, n_repeats=10, random_state=42, n_jobs=None)
sorted_idx = result.importances_mean.argsort()
data = pd.DataFrame(result.importances[sorted_idx])
pd.set_option('display.max_rows', None)
#print(data)# TODO concat all columns
data.index = np.array(GBC.dataset.columns)[sorted_idx]
print(data)
combined = data.stack()
print(combined.shape)
combined = combined.reset_index(level=0)
print(combined)
print(combined.iloc[::-1])
combined.columns = ['features', 'value']
combined['features'] = combined['features'].str.replace('_', ' ')
plt.subplot(1, 2, 2)
g2 = sns.boxplot(data=combined.iloc[::-1], x='value', y='features', color='grey', fliersize=4, linewidth=1.5, whiskerprops={'color':'black'}, flierprops={"marker": "o", 'color':'black', 'linewidth':0, 'markeredgecolor':'black'}, 
capprops={'color':'black'}, boxprops={'color':'grey', 'linewidth':1.5, 'edgecolor':'black'}, medianprops={'color':'#E60234'})
g2.set(xlim=(-.015, .05))
plt.ylabel('')
plt.xlabel('Decrease in accuracy')

plt.title("Permutation Importance (test set)")
plt.subplots_adjust(hspace=.1, top=.5, bottom=0)
fig.tight_layout()
plt.savefig(f"{fig_folder}/Classifier_feature_importance.svg")
plt.show()

Train and assess reduced model:

In [ ]:
# best performing combination
GBC_small = Classifier(final_dataset['2'][['log', 'pLDDT', 'Linker_idx_Suyama',
       'KLD', 'Insert_frequency','Deletion_frequency', 'Mean_ins_len']], n_cross_val=5, title_text='Model trained on selected features')
model_, X_test_, y_test_, X_train_, y_train_, scores_, kf_, scaler_ = GBC_small.split_and_train()
GBC_small.metrics()
scores_['test_recall']

In [ ]:

GBC_small.clf.fit(X_train_, y_train_)
feature_importance = GBC_small.clf.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + 0.5
fig = plt.figure(figsize=(15, 3))
plt.subplot(1, 2, 1)
data = pd.DataFrame([np.array(GBC_small.dataset.columns)[sorted_idx], feature_importance[sorted_idx]]).T
data.columns=['features', 'importance']
data.features = data.features.str.replace('_', ' ')
g = sns.barplot(data= data.iloc[::-1], y='features', x='importance', color='grey', linewidth=1.5, capsize=.2, errcolor="black", edgecolor="black")
g.set(xlim=(0, .35))
plt.ylabel('')
plt.xlabel('Decrease in impurity')
plt.title("Feature Importance (Mean Decrease in Impurity)")

result = permutation_importance(GBC_small.clf, X_test_, y_test_, n_repeats=10, random_state=42, n_jobs=None)
sorted_idx = result.importances_mean.argsort()
data = pd.DataFrame(result.importances[sorted_idx]) 
data.index = np.array(GBC_small.dataset.columns)[sorted_idx]
combined = data.stack()
combined = combined.reset_index(level=0)
combined.columns = ['features', 'value']
print(combined['features'])
combined['features'] = combined['features'].str.replace('_', ' ')
plt.subplot(1, 2, 2)
g2 = sns.boxplot(data=combined.iloc[::-1], x='value', y='features', color='grey', fliersize=4, linewidth=1.5, whiskerprops={'color':'black'}, flierprops={"marker": "o", 'color':'black', 'linewidth':0, 'markeredgecolor':'black'}, 
capprops={'color':'black'}, boxprops={'color':'grey', 'linewidth':1.5, 'edgecolor':'black'}, medianprops={'color':'#E60234'})
g2.set(xlim=(-.03, .05))
plt.ylabel('')
plt.xlabel('Decrease in accuracy')

plt.title("Permutation Importance (test set)")
plt.subplots_adjust(hspace=.1, top=.5, bottom=0)
fig.tight_layout()
plt.savefig(f"{fig_folder}/Classifier_feature_importance_less_features.svg")
plt.show()

## Baselines

In [ ]:

model_predictions = model_complete.predict_proba(X_test_complete)
#print(X_test_complete.shape)
model_predictions = model_predictions[:,1]
#print(model_predictions)
random_baseline = np.random.randint(2, size=len(X_test_complete))
loops_baseline = X_test_complete[:,0]
asa_baseline = X_test_complete[:,23]
hydrophobicity_baseline = X_test_complete[:,28]
print(hydrophobicity_baseline)
baseline_auroc = []
baseline_pr = []
for idx, mod in enumerate([model_predictions, random_baseline, loops_baseline, asa_baseline, hydrophobicity_baseline]):
    baseline_auroc.append(metrics.roc_auc_score(y_test_complete, mod)) 
    baseline_pr.append(metrics.average_precision_score(y_test_complete, mod))

In [ ]:
names = ['Classifier', 'Random baseline', 'Loops', 'Surface exposure', 'Hydrophobicity']
fig = plt.figure(figsize=(5,5))
g = sns.barplot(x=names, y=baseline_auroc, palette=['#87001D', 'grey', 'grey', 'grey', 'grey'], linewidth=1.5, capsize=.2, errcolor="black", edgecolor="black")
g.set_xticklabels(names, rotation=90)
g.set(ylabel='AUROC')
g.set_title(label='AUROC comparison to baselines', fontsize=15)
fig.tight_layout()
plt.savefig(f"{fig_folder}/baselines_auroc.svg")
plt.show()

fig = plt.figure(figsize=(5,5))
g = sns.barplot(x=names, y=baseline_pr, palette=['#174950', 'grey', 'grey', 'grey', 'grey'], linewidth=1.5, capsize=.2, errcolor="black", edgecolor="black")
g.set_xticklabels(names, rotation=90)
g.set(ylabel='Average precision')
g.set_title(label='Precision comparison to baselines', fontsize=15)
fig.tight_layout()
plt.savefig(f"{fig_folder}/baselines_ap.svg")
plt.show()
print(baseline_auroc)
print(baseline_pr)

In [ ]:

model_predictions = model_complete.predict_proba(X_test_complete)
model_predictions = model_predictions[:,1]
test_data_concat = np.c_[model_predictions, X_test_complete]
baseline_auroc = []
baseline_pr = []
for idx in range(test_data_concat.shape[1]):
    baseline_auroc.append(metrics.roc_auc_score(y_test_complete, test_data_concat[:,idx])) 
    baseline_pr.append(metrics.average_precision_score(y_test_complete, test_data_concat[:,idx]))

In [ ]:
names = ['Classifier'] + GBC.dataset.columns.to_list()

fig = plt.figure(figsize=(15,8))
g = sns.barplot(x=names, y=baseline_auroc, color='grey', linewidth=1.5, capsize=.2, errcolor="black", edgecolor="black")
g.set_xticklabels(names, rotation=90)
g.set(ylabel='AUROC')
g.set_title(label='AUROC comparison to baselines', fontsize=15)
g.set(xlim=(-.6, 45.7))
fig.tight_layout()
plt.savefig(f"{fig_folder}/baselines_auroc_full.svg")
plt.show()

fig = plt.figure(figsize=(15,8))
g = sns.barplot(x=names, y=baseline_pr, color='grey', linewidth=1.5, capsize=.2, errcolor="black", edgecolor="black")
g.set_xticklabels(names, rotation=90)
g.set(ylabel='Average precision')
g.set_title(label='Precision comparison to baselines', fontsize=15)
fig.tight_layout()
plt.savefig(f"{fig_folder}/baselines_ap_full.svg")
g.set(xlim=(-.6, 45.7))
plt.show()

In [ ]:
print(GBC.dataset.columns.to_list())